In [1]:
# DS 440 Data Exploration
# Spring 2023
### Instructor: Prof. John Yen
### Group Name: Philly Boys
### Team Members: Andrew cavnaugh, Aman Arora

In [2]:
import pyspark
import csv

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql.functions import array_contains
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import PCA

In [4]:
#Local mode
#Cluster mode
#ss = SparkSession.builder.master("local").appName("Data Analysis Cusltering").getOrCreate()
ss = SparkSession.builder.appName("Data Analysis Cusltering").getOrCreate()

In [5]:
#Read in the data
Scanners_df = ss.read.csv("/storage/home/aka5922/data/complete_Day1_2021profile.csv", header= True, inferSchema=True )
Scanners_df_D2 = ss.read.csv("/storage/home/aka5922/data/complete_Day2_2021profile.csv", header= True, inferSchema=True )

In [6]:
# Need to filter the Extreme Scanners 

In [7]:
multi_port_scanners= Scanners_df.where(col("numports")>1)
multi_port_scanners_D2= Scanners_df_D2.where(col("numports")>1)

In [8]:
ScannersCount_byNumPorts = multi_port_scanners.groupby("numports").count()

In [9]:
ScannersCount_byNumPorts_D2 = multi_port_scanners_D2.groupby("numports").count()

In [10]:
SortedNumPorts_DF= ScannersCount_byNumPorts.orderBy("numports", ascending = False)

In [11]:
SortedNumPorts2_DF= ScannersCount_byNumPorts_D2.orderBy("numports", ascending = False)

In [12]:
max_non_rare_NumPorts= 1000

In [13]:
# Scan the number of ports for 2 ports scanned 
# Save the exteme scanner to csv

In [14]:
extreme_scanners = Scanners_df.where(col("numports") > max_non_rare_NumPorts)
extreme_scanners_D2 = Scanners_df_D2.where(col("numports") > max_non_rare_NumPorts)

In [15]:
#path2_D1="/storage/home/aka5922/Extreme_Scannersone.csv"
#extreme_scanners_D1.write.option("header",True).csv(path2_D1)
#path2_D2="/storage/home/aka5922/Extreme_Scannertwo.csv"
#extreme_scanners_D2.write.option("header",True).csv(path2_D2)

In [16]:
non_extreme_multi_port_scanners = Scanners_df.where(col("numports") <= max_non_rare_NumPorts).where(col("numports") > 1)
non_extreme_multi_port_scanners_D2 = Scanners_df_D2.where(col("numports") <= max_non_rare_NumPorts).where(col("numports") > 1)

In [17]:
# Use One Hot endcoding

In [18]:
# Day1
NEMP_Scanners_df_D1=non_extreme_multi_port_scanners.withColumn("Ports_Array", split(col("ports_scanned_str"), "-") )
# Day2
NEMP_Scanners_df_D2=non_extreme_multi_port_scanners_D2.withColumn("Ports_Array_2", split(col("ports_scanned_str"), "-") )

In [19]:
Ports_Scanned_RDD_D1 = NEMP_Scanners_df_D1.select("Ports_Array").rdd
Ports_Scanned_RDD_D2 = NEMP_Scanners_df_D2.select("Ports_Array_2").rdd

In [20]:
Ports_list_RDD_D1 = Ports_Scanned_RDD_D1.map(lambda row: row[0] )
Ports_list_RDD_D2 = Ports_Scanned_RDD_D2.map(lambda row: row[0] )

In [21]:
Ports_list2_RDD_D1 = Ports_list_RDD_D1.flatMap(lambda x: x )
Ports_list2_RDD_D2 = Ports_list_RDD_D2.flatMap(lambda x: x )

In [22]:
Port_count_RDD_D1 = Ports_list2_RDD_D1.map(lambda x: (x, 1))
Port_count_RDD_D2 = Ports_list2_RDD_D2.map(lambda x: (x, 1))

In [23]:
Port_count_total_RDD_D1 = Port_count_RDD_D1.reduceByKey(lambda x,y: x+y)
Port_count_total_RDD_D2 = Port_count_RDD_D2.reduceByKey(lambda x,y: x+y)

In [24]:
Sorted_Count_Port_RDD = Port_count_total_RDD_D1.map(lambda x: (x[1], x[0])).sortByKey( ascending = False)
Sorted_Count_Port_RDD_D2 = Port_count_total_RDD_D2.map(lambda x: (x[1], x[0])).sortByKey( ascending = False)

In [25]:
top_k_ports = 200
Sorted_Count_Port_RDD.take(top_k_ports)

[(251924, '17136'),
 (251916, '17132'),
 (251531, '17128'),
 (251467, '17140'),
 (251276, '17138'),
 (250654, '17130'),
 (184218, '17142'),
 (184030, '17134'),
 (128832, '23'),
 (111837, '8080'),
 (111822, '80'),
 (49399, '8081'),
 (47547, '2323'),
 (37527, '81'),
 (33323, '49744'),
 (32398, '8443'),
 (31812, '34224'),
 (31715, '34216'),
 (31706, '34228'),
 (31676, '34218'),
 (31537, '33972'),
 (31512, '33964'),
 (31499, '33960'),
 (31493, '34226'),
 (31466, '34220'),
 (31420, '33968'),
 (31394, '33970'),
 (31321, '33962'),
 (30941, '49152'),
 (30171, '7574'),
 (27823, '5555'),
 (26391, '1023'),
 (25946, '52869'),
 (25174, '37215'),
 (25164, '56062'),
 (18810, '34230'),
 (18572, '34222'),
 (18570, '33966'),
 (18485, '33974'),
 (17410, '8181'),
 (17323, '60001'),
 (6782, '445'),
 (4979, '0'),
 (4170, '5353'),
 (4150, '6881'),
 (4027, '8000'),
 (3893, '11211'),
 (3834, '5060'),
 (3798, '8082'),
 (3755, '4000'),
 (3719, '8083'),
 (3653, '443'),
 (3290, '1433'),
 (3131, '30301'),
 (2659, '

In [26]:
Sorted_Count_Port_RDD_D2.take(top_k_ports)

[(256111, '17128'),
 (256071, '17138'),
 (255938, '17132'),
 (255841, '17136'),
 (255675, '17130'),
 (255267, '17140'),
 (187424, '17134'),
 (187143, '17142'),
 (115588, '8080'),
 (115503, '80'),
 (105589, '23'),
 (38254, '62904'),
 (37998, '81'),
 (32353, '34220'),
 (32325, '34226'),
 (32309, '34224'),
 (32293, '34216'),
 (32287, '34218'),
 (32231, '33970'),
 (32098, '33968'),
 (32036, '33960'),
 (32025, '33972'),
 (31993, '33964'),
 (31974, '34228'),
 (31884, '33962'),
 (31417, '8443'),
 (31349, '49152'),
 (30442, '7574'),
 (28626, '52475'),
 (26667, '5555'),
 (26164, '1023'),
 (26016, '2323'),
 (25187, '52869'),
 (24959, '37215'),
 (21922, '8081'),
 (19088, '34222'),
 (19052, '33974'),
 (19045, '34230'),
 (18669, '33966'),
 (17010, '8181'),
 (16917, '60001'),
 (9490, '8550'),
 (8476, '445'),
 (6870, '0'),
 (6401, '30740'),
 (5482, '22'),
 (4913, '17018'),
 (4900, '54046'),
 (4708, '43293'),
 (4307, '13599'),
 (4148, '57628'),
 (4127, '5407'),
 (4110, '64030'),
 (4084, '64284'),
 (40

In [27]:
Sorted_Ports_RDD_D1= Sorted_Count_Port_RDD.map(lambda x: x[1] )
Top_Ports_list_D1 = Sorted_Ports_RDD_D1.take(top_k_ports)


In [28]:
Sorted_Ports_RDD_D2= Sorted_Count_Port_RDD_D2.map(lambda x: x[1] )
Top_Ports_list_D2 = Sorted_Ports_RDD_D2.take(top_k_ports)

In [29]:
#Unionise the Top ports day 1 and day 2

In [30]:
Top_Ports_D1D2 = Top_Ports_list_D1
for i in range(0,199):
    same_port_found = False
    for j in range(0,199):
        if Top_Ports_D1D2[j] == Top_Ports_list_D2[i]:
            same_port_found = True
    if same_port_found == False:
        Top_Ports_D1D2.append(Top_Ports_list_D2[i])

In [31]:
Top_Ports_list_D1[199]

'5601'

In [32]:
Top_Ports_list_D2[199]

'12345'

In [33]:
from pyspark.sql.functions import array_contains

In [34]:
top_k_ports_D1D2= len(Top_Ports_D1D2)

In [35]:
# Initialize NEMP_Scanners2_df
NEMP_Scanners2_df_D1 = NEMP_Scanners_df_D1
for i in range(0, top_k_ports_D1D2):
    # "Port" + Top_Ports_list[i]  is the name of each new feature created through One Hot Encoding Top_Ports_list
    NEMP_Scanners3_df_D1 = NEMP_Scanners2_df_D1.withColumn("Port" + Top_Ports_D1D2[i], array_contains("Ports_Array", Top_Ports_D1D2[i]))
    NEMP_Scanners2_df_D1 = NEMP_Scanners3_df_D1

In [36]:
# Initialize NEMP_Scanners2_df
NEMP_Scanners2_df_D2 = NEMP_Scanners_df_D2
for i in range(0, top_k_ports_D1D2):
    # "Port" + Top_Ports_list[i]  is the name of each new feature created through One Hot Encoding Top_Ports_list
    NEMP_Scanners3_df_D2 = NEMP_Scanners2_df_D2.withColumn("Port" + Top_Ports_D1D2[i], array_contains("Ports_Array_2", Top_Ports_D1D2[i]))
    NEMP_Scanners2_df_D2 = NEMP_Scanners3_df_D2

In [37]:
input_features = [ ]
for i in range(0, top_k_ports_D1D2 ):
    input_features.append( "Port"+ Top_Ports_D1D2[i] )

In [38]:
va_D1 = VectorAssembler().setInputCols(input_features).setOutputCol("features")
va_D2 = VectorAssembler().setInputCols(input_features).setOutputCol("features")

In [39]:
data_D1= va_D1.transform(NEMP_Scanners2_df_D1)
data_D2= va_D2.transform(NEMP_Scanners2_df_D2)

In [40]:
total_clusters_D1D2 = top_k_ports_D1D2
km = KMeans(featuresCol= "features", predictionCol="prediction").setK(top_k_ports_D1D2).setSeed(123)
km.explainParams()

'distanceMeasure: the distance measure. Supported options: \'euclidean\' and \'cosine\'. (default: euclidean)\nfeaturesCol: features column name. (default: features, current: features)\ninitMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)\ninitSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)\nk: The number of clusters to create. Must be > 1. (default: 2, current: 272)\nmaxIter: max number of iterations (>= 0). (default: 20)\npredictionCol: prediction column name. (default: prediction, current: prediction)\nseed: random seed. (default: 6185308224697046630, current: 123)\ntol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)\nweightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)'

In [41]:
kmModel_D1=km.fit(data_D1)

In [42]:
kmModel_D2=km.fit(data_D2)

In [43]:
predictions_D1 = kmModel_D1.transform(data_D1)

In [44]:
predictions_D2 = kmModel_D2.transform(data_D2)

In [45]:
Cluster1_df_D1=predictions_D1.where(col("prediction")==0)

In [46]:
Cluster1_df_D2=predictions_D2.where(col("prediction")==0)

In [47]:
summary_D1 = kmModel_D1.summary

In [48]:
summary_D2 = kmModel_D2.summary

In [49]:
summary_D1.clusterSizes

[2872,
 2908,
 12588,
 541,
 2318,
 9798,
 584,
 1914,
 9243,
 1010,
 7262,
 7802,
 3358,
 64332,
 383,
 390,
 2927,
 3280,
 598,
 458,
 594,
 9511,
 1241,
 8471,
 4561,
 2540,
 1912,
 1924,
 24991,
 1003,
 756,
 782,
 775,
 3829,
 7658,
 955,
 62632,
 545,
 8301,
 2780,
 948,
 3938,
 829,
 18744,
 7504,
 2949,
 6652,
 620,
 2386,
 1350,
 6574,
 2248,
 2783,
 7589,
 94,
 2676,
 504,
 1074,
 2071,
 491,
 5021,
 232,
 288,
 1556,
 1514,
 2061,
 423,
 5186,
 2502,
 480,
 491,
 7384,
 983,
 8866,
 2570,
 90,
 16,
 3286,
 864,
 597,
 1876,
 1126,
 6587,
 724,
 4307,
 2159,
 2560,
 1396,
 3641,
 2655,
 3885,
 590,
 451,
 881,
 2065,
 343,
 1519,
 3006,
 991,
 2480,
 2637,
 1215,
 774,
 3046,
 1747,
 796,
 1921,
 664,
 3621,
 1380,
 675,
 3479,
 3843,
 3740,
 2658,
 1286,
 1947,
 851,
 1007,
 549,
 1596,
 939,
 1763,
 1863,
 516,
 570,
 1235,
 608,
 1373,
 2520,
 615,
 506,
 472,
 487,
 499,
 1198,
 4281,
 6567,
 4295,
 7643,
 487,
 3104,
 1209,
 7775,
 2336,
 914,
 918,
 628,
 471,
 386,
 12

In [50]:
summary_D2.clusterSizes

[9752,
 10927,
 8841,
 1780,
 5320,
 836,
 8608,
 4747,
 867,
 2898,
 1231,
 60390,
 506,
 28481,
 35,
 3180,
 6655,
 680,
 2377,
 1273,
 2688,
 3661,
 5884,
 1634,
 682,
 656,
 539,
 539,
 1066,
 70942,
 909,
 822,
 568,
 5180,
 2333,
 2051,
 4259,
 9650,
 3211,
 3181,
 1577,
 981,
 1170,
 9599,
 8661,
 5712,
 860,
 1237,
 344,
 4290,
 809,
 5109,
 2485,
 11285,
 6710,
 2068,
 1251,
 3974,
 3440,
 703,
 899,
 756,
 3194,
 3636,
 2404,
 3014,
 1099,
 4352,
 519,
 1113,
 2112,
 3941,
 3737,
 6636,
 4381,
 1595,
 681,
 330,
 7541,
 8613,
 2648,
 1045,
 560,
 8118,
 86,
 3849,
 677,
 2817,
 584,
 665,
 3478,
 3439,
 722,
 359,
 2618,
 5174,
 6616,
 6359,
 465,
 6476,
 1169,
 2214,
 356,
 8632,
 2575,
 617,
 1739,
 694,
 3215,
 1038,
 1370,
 3873,
 920,
 2262,
 1182,
 3163,
 3534,
 2607,
 82,
 2068,
 4208,
 1036,
 3165,
 1055,
 1008,
 3523,
 1457,
 576,
 743,
 2757,
 1788,
 2038,
 2311,
 401,
 521,
 1984,
 1347,
 1919,
 9731,
 2328,
 4161,
 231,
 3818,
 3590,
 1273,
 671,
 947,
 2840,
 444

In [51]:
evaluator = ClusteringEvaluator()

In [52]:
silhouette_D1 = evaluator.evaluate(predictions_D1)
silhouette_D2 = evaluator.evaluate(predictions_D2)

In [53]:
centers_D1 = kmModel_D1.clusterCenters()

In [54]:
centers_D2 = kmModel_D2.clusterCenters()

In [55]:
import pandas as pd
import numpy as np
import math

In [ ]:
# Define columns of the Pandas dataframe
column_list = ['cluster ID', 'size'] #mirai ratio
for feature in input_features:
    column_list.append(feature)
clusters_summary_df_D1 = pd.DataFrame( columns = column_list )
clusters_summary_df_D2 = pd.DataFrame( columns = column_list )

#D1
for i in range(0, total_clusters_D1D2):
    cluster_i = predictions_D1.where(col('prediction')==i)
    cluster_i_size = cluster_i.count()
    cluster_row = [i, cluster_i_size] #cluster_i_mirai_ratio
    for j in range(0, len(input_features)):
        cluster_row.append(centers_D1[i][j])
    clusters_summary_df_D1.loc[i]= cluster_row

In [ ]:
#D2
for i in range(0, total_clusters):
    cluster_i = predictions_D1.where(col('prediction')==i)
    cluster_i_size = cluster_i.count()
    cluster_row = [i, cluster_i_size]
    for j in range(0, len(input_features)):
        cluster_row.append(centers_D2[i][j])
    clusters_summary_df_D2.loc[i]= cluster_row

In [ ]:
path4_D1= "/storage/home/aka5922/Data clustered/Clusters_Summary_D1_2.csv"
clusters_summary_df_D1.to_csv(path4_D1, header=True)
path4_D2= "/storage/home/aka5922/Data clustered/Clusters_Summary_D2_2.csv"
clusters_summary_df_D2.to_csv(path4_D2, header=True)

In [ ]:
J_Nearest_Clusters_D2_D1_df = pd.DataFrame(columns=["D2Cindex", "D2Csize", "D1NearestCindex", "D1Csize", "Similarity"])
# Cluster Centers of Day 2
centers_D2 = centers_day1
# Cluster Centers of Day 1
centers_D1 = centers_day2
i = 0
for center2 in centers_D2:
    j = 0
    # Initialize shortest_dist
    highest_similarity = 0
    closest_cluster = -1
    for center1 in centers_D1:
        # skip clusters whose cluster centers are all zeros
        if np.sum(center1) != 0:
            fuzzy_intersection = np.minimum(center1, center2)
            fuzzy_union = np.maximum(center1, center2)
            cluster_i_j_similarity = np.sum(fuzzy_intersection)/np.sum(fuzzy_union)
            if cluster_i_j_similarity > highest_similarity:
                highest_similarity = cluster_i_j_similarity
                closest_cluster = j
        j = j+1
    if closest_cluster != -1:
        J_Nearest_Clusters_D2_D1_df.loc[i]=[ i, summary_day2.clusterSizes[i], closest_cluster, summary_day1.clusterSizes[closest_cluster], highest_similarity]
    else:
        J_Nearest_Clusters_D2_D1_df.loc[i]=[ i, summary_day2.clusterSizes[i], -1, 0, 0 ]
    i = i+1

In [ ]:
path5= "/storage/home/aka5922/Clusters_Summary_J_Nearest_Clusters_D2_D1_df_1.csv"
clusters_summary_df.to_csv(path5, header=True)